In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install beautifulsoup4 

In [ ]:
pip install transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 42.5 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install emoji

     |████████████████████████████████| 184 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=ea0d6e47d9c803ba3459e00319fcfbd22c175f1e7df2e2678329cf9bedbed517
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji


In [ ]:
import os
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [ ]:
%cd 

/root


In [ ]:
df_train = pd.read_csv("../content/drive/Shareddrives/Soft Computing Lab/Dataset/Go emotion dataset/data/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv("../content/drive/Shareddrives/Soft Computing Lab/Dataset/Go emotion dataset/data/test.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [ ]:
df_train['List of classes'] = df_train['Class'].apply(lambda x: x.split(','))
df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))
df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))

In [ ]:
with open('../content/drive/Shareddrives/Soft Computing Lab/Dataset/Go emotion dataset/data/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

In [ ]:
emotion_file = open("../content/drive/Shareddrives/Soft Computing Lab/Dataset/Go emotion dataset/data/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [ ]:
df_train['Emotions'] = df_train['List of classes'].apply(idx2class)
df_dev['Emotions'] = df_dev['List of classes'].apply(idx2class)

In [ ]:
def EmotionMapping(emotion_list):
    map_list = []
    
    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['worry']:
            map_list.append('worry')
        if i in ekman_mapping['happiness']:
            map_list.append('happiness')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')
            
    return map_list
  

In [ ]:
df_train['Mapped Emotions'] = df_train['Emotions'].apply(EmotionMapping)
df_dev['Mapped Emotions'] = df_dev['Emotions'].apply(EmotionMapping)

In [ ]:
df_train['anger'] = np.zeros((len(df_train),1))
df_train['worry'] = np.zeros((len(df_train),1))
df_train['happiness'] = np.zeros((len(df_train),1))
df_train['sadness'] = np.zeros((len(df_train),1))
df_train['surprise'] = np.zeros((len(df_train),1))
df_train['neutral'] = np.zeros((len(df_train),1))

df_dev['anger'] = np.zeros((len(df_dev),1))
df_dev['worry'] = np.zeros((len(df_dev),1))
df_dev['happiness'] = np.zeros((len(df_dev),1))
df_dev['sadness'] = np.zeros((len(df_dev),1))
df_dev['surprise'] = np.zeros((len(df_dev),1))
df_dev['neutral'] = np.zeros((len(df_dev),1))

In [ ]:
for i in ['anger',  'worry', 'happiness', 'sadness', 'surprise','neutral']:
    df_train[i] = df_train['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    df_dev[i] = df_dev['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [ ]:
df_train.head(100)

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,worry,happiness,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[worry],0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Damn for that much you may as well use a mobil...,27,efch84n,[27],1,[neutral],[neutral],0,0,0,0,0,1
96,I think the 90 day rule applies to increases o...,7,efae8sy,[7],1,[curiosity],[surprise],0,0,0,0,1,0
97,If they bolt its a feature.,27,edpcbde,[27],1,[neutral],[neutral],0,0,0,0,0,1
98,So this means the people who have debt can see...,7,ees00ae,[7],1,[curiosity],[surprise],0,0,0,0,1,0


In [ ]:
df_dev.head(100)

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,worry,happiness,sadness,surprise,neutral
0,I’m really sorry about your situation :( Altho...,25,eecwqtt,[25],1,[sadness],[sadness],0,0,0,1,0,0
1,It's wonderful because it's awful. At not with.,0,ed5f85d,[0],1,[admiration],[happiness],0,0,1,0,0,0
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3,[13],1,[excitement],[happiness],0,0,1,0,0,0
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1,[15],1,[gratitude],[happiness],0,0,1,0,0,0
4,They got bored from haunting earth for thousan...,27,eem5uti,[27],1,[neutral],[neutral],0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,BOOM BOOM BOOM (not Bitton related),27,eeglc1u,[27],1,[neutral],[neutral],0,0,0,0,0,1
96,Because of this bad situation some people are ...,14,ed99kw8,[14],1,[fear],[worry],0,1,0,0,0,0
97,That’s how you know they’re happy.,"13,27",efb2j1c,"[13, 27]",2,"[excitement, neutral]","[happiness, neutral]",0,0,1,0,0,1
98,Incoming 20m a year contract [NAME] !!!,27,ef6bmgl,[27],1,[neutral],[neutral],0,0,0,0,0,1


In [ ]:
content = list(df_train['Text']);
emotions  = list(df_train['Mapped Emotions'])
print(emotions)
emotion = [emo[0] for emo in emotions];


[['neutral'], ['neutral'], ['anger'], ['worry'], ['anger'], ['surprise'], ['happiness'], ['happiness', 'happiness'], ['happiness'], ['neutral'], ['surprise'], ['happiness', 'happiness'], ['neutral'], ['happiness'], ['anger'], ['anger', 'sadness'], ['happiness'], ['anger'], ['neutral'], ['surprise', 'surprise'], ['surprise', 'sadness', 'neutral'], ['sadness'], ['neutral'], ['neutral'], ['neutral'], ['anger'], ['neutral'], ['sadness', 'sadness'], ['happiness'], ['neutral'], ['anger'], ['surprise'], ['neutral'], ['anger', 'surprise'], ['surprise'], ['happiness'], ['neutral'], ['happiness'], ['sadness'], ['neutral'], ['happiness', 'happiness'], ['happiness', 'happiness'], ['sadness', 'neutral'], ['neutral'], ['surprise', 'happiness'], ['anger'], ['happiness'], ['neutral'], ['neutral'], ['neutral'], ['neutral'], ['neutral'], ['happiness'], ['neutral'], ['happiness', 'happiness'], ['anger'], ['neutral'], ['neutral'], ['neutral'], ['happiness'], ['happiness', 'happiness'], ['sadness'], ['neut

In [ ]:
content1 = list(df_dev['Text']);
emotions1  = list(df_dev['Mapped Emotions'])
print(emotions)
emotion1 = [emo[0] for emo in emotions1];
print(len(content1));

[['neutral'], ['neutral'], ['anger'], ['worry'], ['anger'], ['surprise'], ['happiness'], ['happiness', 'happiness'], ['happiness'], ['neutral'], ['surprise'], ['happiness', 'happiness'], ['neutral'], ['happiness'], ['anger'], ['anger', 'sadness'], ['happiness'], ['anger'], ['neutral'], ['surprise', 'surprise'], ['surprise', 'sadness', 'neutral'], ['sadness'], ['neutral'], ['neutral'], ['neutral'], ['anger'], ['neutral'], ['sadness', 'sadness'], ['happiness'], ['neutral'], ['anger'], ['surprise'], ['neutral'], ['anger', 'surprise'], ['surprise'], ['happiness'], ['neutral'], ['happiness'], ['sadness'], ['neutral'], ['happiness', 'happiness'], ['happiness', 'happiness'], ['sadness', 'neutral'], ['neutral'], ['surprise', 'happiness'], ['anger'], ['happiness'], ['neutral'], ['neutral'], ['neutral'], ['neutral'], ['neutral'], ['happiness'], ['neutral'], ['happiness', 'happiness'], ['anger'], ['neutral'], ['neutral'], ['neutral'], ['happiness'], ['happiness', 'happiness'], ['sadness'], ['neut

In [ ]:
co =[];
for c in content1:
    content.append(c);
for e  in emotion1:
  emotion.append(e);
print(len(content));
print(len(emotion));
print(len(df_dev));
print(len(co))
data ={
    'content' : content,
    'sentiment' : emotion
};

48837
48837
5427
0


In [ ]:
new_df = pd.DataFrame(data, columns=['content','sentiment']);
print(len(new_df));

48837


In [ ]:
new_df.drop_duplicates(subset = "content" , keep = False ,  inplace = True);
print(len(new_df));
print(new_df);

48480
                                                 content  sentiment
0      My favourite food is anything I didn't have to...    neutral
1      Now if he does off himself, everyone will thin...    neutral
2                         WHY THE FUCK IS BAYLESS ISOING      anger
3                            To make her feel threatened      worry
4                                 Dirty Southern Wankers      anger
...                                                  ...        ...
48832  Thanks. I was diagnosed with BP 1 after the ho...  happiness
48833                             Well that makes sense.  happiness
48834                                Daddy issues [NAME]    neutral
48835  So glad I discovered that subreddit a couple m...  happiness
48836  Had to watch "Elmo in Grouchland" one time too...    neutral

[48480 rows x 2 columns]


In [ ]:
new_df = new_df.sample(frac = 1);
print(new_df);
print(len(new_df));

                                                 content  sentiment
5105   Character and emotions are not the reigning ch...      anger
48198  I used to know a guy back in HS, and college. ...    neutral
14460  You're right. Lets see what Fox News Analyst L...  happiness
453                                      You are sexy af  happiness
17689  Turn in them. Who cares if they’re your cousin...      anger
...                                                  ...        ...
3946   As a kid, I was way more excited to see the [N...  happiness
41294                       You don't know what fact is?    neutral
38167  The woman types like she isn’t a native speake...    sadness
27542      My DVR did the same thing as yours Damn them.      anger
15368  Sorry to hear that, mate. Please get some prof...  happiness

[48480 rows x 2 columns]
48480


In [ ]:
new_df.to_csv('Go Emotion preprocessed.csv');

In [ ]:
print(new_df);

                                                 content  sentiment
5105   Character and emotions are not the reigning ch...      anger
48198  I used to know a guy back in HS, and college. ...    neutral
14460  You're right. Lets see what Fox News Analyst L...  happiness
453                                      You are sexy af  happiness
17689  Turn in them. Who cares if they’re your cousin...      anger
...                                                  ...        ...
3946   As a kid, I was way more excited to see the [N...  happiness
41294                       You don't know what fact is?    neutral
38167  The woman types like she isn’t a native speake...    sadness
27542      My DVR did the same thing as yours Damn them.      anger
15368  Sorry to hear that, mate. Please get some prof...  happiness

[48480 rows x 2 columns]


In [ ]:
from google.colab import files
files.download('Go Emotion preprocessed.csv'); 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_train.drop(df_train[df_train['neutral'] == 1].index, inplace=True)
df_dev.drop(df_dev[df_dev['neutral'] == 1].index, inplace=True)
df_train.drop(df_train[df_train['disgust'] == 1].index, inplace=True)
df_dev.drop(df_dev[df_dev['disgust'] == 1].index, inplace=True)

In [ ]:
df_train.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions', 'neutral', 'disgust'], axis=1, inplace=True)
df_dev.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions', 'neutral','disgust'], axis=1, inplace=True)

##  amar edit no need to run


In [ ]:
df_train.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)
df_dev.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)
display(df_train);

,Text,ID,anger,worry,happiness,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,eebbqej,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",ed00q6i,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,eezlygj,1,0,0,0,0,0
3,To make her feel threatened,ed7ypvh,0,1,0,0,0,0
4,Dirty Southern Wankers,ed0bdzj,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,edsb738,0,0,1,0,0,0
43406,Always thought that was funny but is it a refe...,ee7fdou,0,0,0,0,1,0
43407,What are you talking about? Anything bad that ...,efgbhks,1,0,0,0,0,0
43408,"More like a baptism, with sexy results!",ed1naf8,0,0,1,0,0,0


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [ ]:
 df_train['Text'] = df_train['Text'].apply(text_preprocessing_pipeline)
 df_dev['Text'] = df_dev['Text'].apply(text_preprocessing_pipeline)

In [ ]:
df_train.reset_index(drop=True).to_csv("train.csv", index=False)
df_dev.reset_index(drop=True).to_csv("val.csv", index=False)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_dev = df_dev.reset_index(drop=True)

In [ ]:
df_train.head()

,Text,ID,anger,worry,happiness,sadness,surprise,neutral
0,my favorite food is anything i did not have to...,eebbqej,0,0,0,0,0,1
1,now if he does off himself everyone will think...,ed00q6i,0,0,0,0,0,1
2,why the fuck is bayless isoing,eezlygj,1,0,0,0,0,0
3,to make her feel threatened,ed7ypvh,0,1,0,0,0,0
4,dirty southern wankers,ed0bdzj,1,0,0,0,0,0


In [ ]:
print(df_train.shape)
print(df_dev.shape)

(43410, 8)
(5427, 8)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 6
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [ ]:
target_cols = [col for col in df_train.columns if col not in ['Text', 'ID']]
target_cols

['anger', 'worry', 'happiness', 'sadness', 'surprise', 'neutral']

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(df_dev, tokenizer, MAX_LEN)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,6)
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

model = BERTClass()
model.to(device);

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  0.7140201330184937
Epoch: 0, Loss:  0.4393249452114105
Epoch: 0, Loss:  0.26133108139038086
Epoch: 0, Loss:  0.2556045949459076
Epoch: 0, Loss:  0.2212761640548706
Epoch: 0, Loss:  0.3129158914089203
Epoch: 1, Loss:  0.14921998977661133
Epoch: 1, Loss:  0.18872198462486267
Epoch: 1, Loss:  0.2241010069847107
Epoch: 1, Loss:  0.32654255628585815
Epoch: 1, Loss:  0.20702092349529266
Epoch: 1, Loss:  0.097335085272789
Epoch: 2, Loss:  0.19929289817810059
Epoch: 2, Loss:  0.14498507976531982
Epoch: 2, Loss:  0.31108707189559937
Epoch: 2, Loss:  0.2050774097442627
Epoch: 2, Loss:  0.22674314677715302
Epoch: 2, Loss:  0.14868539571762085
Epoch: 3, Loss:  0.059961821883916855
Epoch: 3, Loss:  0.07468922436237335
Epoch: 3, Loss:  0.17128005623817444
Epoch: 3, Loss:  0.11540929973125458
Epoch: 3, Loss:  0.14063841104507446
Epoch: 3, Loss:  0.14992448687553406
Epoch: 4, Loss:  0.11536004394292831
Epoch: 4, Loss:  0.08715811371803284
Epoch: 4, Loss:  0.1124739795923233
Epoch: 4, 

In [ ]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.5843007186290768
F1 Score (Micro) = 0.6639288158795346
F1 Score (Macro) = 0.6233316721267506


In [ ]:
torch.save(model.state_dict(), 'model.bin')

In [ ]:
from transformers import AutoConfig, AutoModel

In [ ]:
#config = AutoConfig.from_pretrained('bert-base-uncased')
#model =  AutoModel.from_config(config)

In [ ]:
#PATH = ""
#model.load_state_dict(torch.load(PATH))